# Scheduler K8S Test

Assumes you have a Kind cluster running with metallb. No other dependencies required.
You can use [Seldon Ansible Kind Playbook](https://github.com/SeldonIO/ansible-k8s-collection/blob/master/playbooks/kind.yaml)

## Setup

* `make kind-image-install-all`
* `make deploy` 


In [61]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP

In [62]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP

In [63]:
!grpcurl -d '{"model":{"name":"iris","uri":"gs://seldon-models/mlserver/iris",\
                      "requirements":["sklearn"],\
                      "memoryBytes":50000,\
                      "replicas":2}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

{
  
}


In [81]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "modelReplicaState": {
    "0": "Loaded",
    "1": "Loaded"
  }
}


In [82]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x563904c1d4f0)
*   Trying 172.18.255.1...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x563904c1d4f0)
* Connected to 172.18.255.1 (172.18.255.1) port 80 (#0)







* upload completely sent off: 94 out of 94 bytes







* Connection #0 to host 172.18.255.1 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"b8ee6cf5-6a0c-4ad7-aca6-ad728b76a201","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [13]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "100000",
      "availableMemoryBytes": "50000"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "50000"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    }
  ]
}


In [39]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


In [40]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x56155dd324f0)
*   Trying 172.18.255.1...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x56155dd324f0)
* Connected to 172.18.255.1 (172.18.255.1) port 80 (#0)







* upload completely sent off: 94 out of 94 bytes






* Closing connection 0


In [16]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

ERROR:
  Code: FailedPrecondition
  Message: Failed to find model iris


In [17]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    }
  ]
}
